# Model : LTSM

In [2]:
import pandas as pd
import numpy as np
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

pd.options.mode.chained_assignment = None 

2022-02-22 13:19:54.333435: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-22 13:19:54.333476: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Load data
df = pd.read_csv("../data/ohlcv_m6.csv")
print(df[df["Symbol"]=="CARR"])

              Date Symbol       Open       High        Low      Close   Volume
109537  2020-03-19   CARR  11.766294  11.766294  11.766294  11.766294      100
109538  2020-03-20   CARR  12.746820  12.746820  11.766295  12.501689     2300
109539  2020-03-23   CARR  12.501688  12.501688  11.276032  11.315253    18700
109540  2020-03-24   CARR  12.648768  12.648768  11.276034  11.472138    96100
109541  2020-03-25   CARR  11.570191  13.188056  11.570191  13.188056    19200
...            ...    ...        ...        ...        ...        ...      ...
110017  2022-02-11   CARR  47.040001  47.220001  44.990002  45.500000  5862500
110018  2022-02-14   CARR  45.520000  45.779999  44.290001  44.790001  6410400
110019  2022-02-15   CARR  45.560001  45.860001  45.340000  45.750000  4648300
110020  2022-02-16   CARR  45.459999  45.910000  45.000000  45.650002  4210100
110021  2022-02-17   CARR  45.160000  45.500000  44.459999  44.580002  4201700

[485 rows x 7 columns]


In [10]:
def make_predictions(df):
    # PARAMETERS
    split_percent = 1 # Split ratio for the train/test split. Set to 1 if we do not test the model
    look_back = 15 # Look back for the LTSM
    num_epochs = 50 # Number of epochs for the LTSM
    num_prediction = 30 # Make forecast for the next month
    
    # Forecast dates
    last_date = df['Date'].values[-1]
    forecast_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    
    # This dataframe stores the forecasts of the next month for every symbols
    res = pd.DataFrame()
    res["Date"] = forecast_dates
    
    # Get list of assets
    assets = pd.unique(df["Symbol"].values.ravel())

    for symbol in assets:
        tf.keras.backend.clear_session()
        print(symbol)
        
        # Get data. Keep only Close values
        dataframe = df[df["Symbol"]==symbol]
        #print("##############",len(dataframe))
        dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d")
        dataframe.set_axis(dataframe['Date'], inplace=True)
        dataframe.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
        close_data = dataframe["Close"].values.reshape(-1,1)
        
        #print("##############",len(close_data))
        
        # Split
        split = int(split_percent*len(close_data))
        close_train = close_data[:split]
        #close_test = close_data[split:]
        date_train = dataframe['Date'][:split]
        #date_test = dataframe['Date'][split:]
        
        
        train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)
        #test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

        # Create the model (LTSM)
        model = Sequential()
        model.add(
            LSTM(10,
                activation='relu',
                input_shape=(look_back,1))
        )
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        
        #dot_img_file = symbol+"_model.png"
        #tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

        
        # Fit the model
        model.fit(train_generator, epochs=num_epochs, verbose=2)
        
        # Predictions
        #prediction = model.predict(test_generator)

        close_train = close_train.reshape((-1))
        #close_test = close_test.reshape((-1))
        #prediction = prediction.reshape((-1))
                
        # Make prediction for the next month
        close_data = close_data.reshape((-1))

       
        prediction_list = close_data[-look_back:]

        for _ in range(num_prediction):
            x = prediction_list[-look_back:]
            x = x.reshape((1, look_back, 1))
            out = model.predict(x)[0][0]
            prediction_list = np.append(prediction_list, out)
        forecast = prediction_list[look_back-1:]

        
        # Transform results as dataframes
        df_train = pd.DataFrame({"Date":date_train,"Train" : close_train})
        #df_test = pd.DataFrame({"Date":date_test,"Test" : close_test})
        #df_prediction = pd.DataFrame({"Date":date_test[look_back:],"Prediction" : prediction})
        df_forecast = pd.DataFrame({"Date":forecast_dates,"Forecast" : forecast})
        res[symbol] = forecast
        
        
    return res#df_train, df_test, df_prediction, df_forecast
        

In [11]:
#df_train, df_test, df_prediction, df_forecast = make_predictions(df)
forecasts = make_predictions(df)

CARR
[[11.76629448]
 [12.50168896]
 [11.31525326]
 [11.4721384 ]
 [13.18805599]
 [15.14910412]
 [14.26663208]
 [15.59034061]
 [16.91404724]
 [14.75689507]
 [13.02136612]
 [16.59047699]
 [15.13929939]
 [14.95300102]
 [13.98228073]
 [14.49215317]
 [13.71753979]
 [13.5998764 ]
 [13.09980774]
 [12.6389637 ]
 [13.47240734]
 [14.16858101]
 [14.40390682]
 [14.50195885]
 [15.41384602]
 [15.93352509]
 [16.47281265]
 [16.66892052]
 [17.29645348]
 [17.36509132]
 [16.44339752]
 [16.08060265]
 [15.65897846]
 [15.97274685]
 [16.09041214]
 [18.00243187]
 [17.9926281 ]
 [17.45333862]
 [16.11982727]
 [16.22768211]
 [16.38456535]
 [17.20820618]
 [17.42392349]
 [18.36522484]
 [19.07120514]
 [18.67899132]
 [19.1300354 ]
 [19.60068512]
 [19.21828651]
 [20.07133865]
 [21.39504623]
 [20.94400597]
 [21.78725433]
 [22.23829842]
 [23.10115814]
 [23.89538956]
 [22.75797462]
 [21.76764679]
 [20.64984894]
 [21.12050438]
 [21.54212379]
 [21.64017677]
 [20.8459549 ]
 [20.83614922]
 [21.03225517]
 [22.29713058]
 [21.

2022-02-22 13:33:21.516322: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-22 13:33:21.517849: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-22 13:33:21.517862: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-22 13:33:21.517885: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kfdell): /proc/driver/nvidia/version does not exist
2022-02-22 13:33:21.518092: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

24/24 - 1s - loss: 1011.6743


In [ ]:
#forecasts.plot(x="Date", y=list(forecasts.columns[1:]), label=list(forecasts.columns[1:]))
#plt.show()
forecasts.to_csv("forecasts.csv")